# Iniciando

In [ ]:
!pip install sympy
!pip install ultralytics

from ultralytics import YOLO

In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount= True)

Mounted at /content/drive


# Criar dataset

In [ ]:
# TILES
import os
import re
import shutil
import random
from pathlib import Path
from collections import Counter, defaultdict
import yaml
from PIL import Image

images_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Sao_Carlos_imagens"
labels_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Sao_Carlos_finalAnnotations"
output_root = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2"
tiles_root = os.path.join(output_root, "tiles")
train_root = os.path.join(tiles_root, "train")
test_root = os.path.join(tiles_root, "test")
os.makedirs(train_root, exist_ok=True)
os.makedirs(test_root, exist_ok=True)

random_seed = 42
random.seed(random_seed)

tile_size = 1024
overlap = 0.2
keep_negative_tiles = True

image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

def numeric_key(filename):
    name = Path(filename).stem
    m = re.match(r"^(\d+)$", name)
    if m:
        return int(m.group(1))
    nums = re.findall(r"\d+", name)
    return int(nums[0]) if nums else name

def list_images(dirpath):
    return [f for f in os.listdir(dirpath) if Path(f).suffix.lower() in image_extensions]

all_images = sorted(list_images(images_dir), key=numeric_key)
if not all_images:
    raise SystemExit("no images found in images_dir")

def ensure_label_file(labels_dir, stem):
    src = os.path.join(labels_dir, stem + ".txt")
    if not os.path.exists(src):
        open(src, "a").close()
    return src

def read_yolo_labels(path):
    boxes = []
    try:
        with open(path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue
                cls = int(float(parts[0]))

                # Classes a serem removidas: 'lixo' (5) e 'Monte_de_objetos' (8)
                if cls in [5, 8]:
                    continue

                if cls > 5:
                    cls -= 1

                x_center = float(parts[1])
                y_center = float(parts[2])
                w = float(parts[3])
                h = float(parts[4])
                boxes.append((cls, x_center, y_center, w, h))
    except:
        pass
    return boxes

def yolo_to_abs(box, img_w, img_h):
    cls, x_c, y_c, w, h = box
    x_c *= img_w
    y_c *= img_h
    w *= img_w
    h *= img_h
    x1 = x_c - w / 2
    y1 = y_c - h / 2
    x2 = x_c + w / 2
    y2 = y_c + h / 2
    return cls, x1, y1, x2, y2

def abs_to_yolo(cls, x1, y1, x2, y2, tile_w, tile_h):
    w = max(1e-6, x2 - x1)
    h = max(1e-6, y2 - y1)
    x_c = (x1 + x2) / 2 / tile_w
    y_c = (y1 + y2) / 2 / tile_h
    return f"{cls} {x_c:.6f} {y_c:.6f} {w/tile_w:.6f} {h/tile_h:.6f}"

def dominant_class_for_image(boxes):
    if not boxes:
        return -1
    counter = Counter([b[0] for b in boxes])
    return counter.most_common(1)[0][0]

# Coleta metadados das imagens
image_meta = []
for img_name in all_images:
    stem = Path(img_name).stem
    lbl = ensure_label_file(labels_dir, stem)
    img_path = os.path.join(images_dir, img_name)
    try:
        with Image.open(img_path) as im:
            w, h = im.size
    except:
        continue
    boxes = read_yolo_labels(lbl)
    dom = dominant_class_for_image(boxes)
    image_meta.append({"img": img_name, "stem": stem, "w": w, "h": h, "boxes": boxes, "dom": dom})

# Estratificação por classe dominante
groups = defaultdict(list)
for m in image_meta:
    groups[m["dom"]].append(m)

train_images = []
test_images = []
for dom, items in groups.items():
    random.shuffle(items)
    split = int(0.8 * len(items))
    train_images.extend(items[:split])
    test_images.extend(items[split:])

# Função para criar tiles
def make_tiles_for_image(meta, out_dir, tile_size=1024, overlap=0.2, keep_neg=False):
    img_name = meta["img"]
    stem = meta["stem"]
    img_path = os.path.join(images_dir, img_name)
    abs_boxes = [yolo_to_abs(b, meta["w"], meta["h"]) for b in meta["boxes"]]
    stride = int(tile_size * (1 - overlap))
    with Image.open(img_path) as im:
        W, H = im.size
        x_starts = list(range(0, max(1, W - tile_size + 1), stride))
        y_starts = list(range(0, max(1, H - tile_size + 1), stride))
        if x_starts and x_starts[-1] + tile_size < W:
            x_starts.append(W - tile_size)
        if y_starts and y_starts[-1] + tile_size < H:
            y_starts.append(H - tile_size)
        for xi, x in enumerate(x_starts):
            for yi, y in enumerate(y_starts):
                box_list = []
                tile_x1, tile_y1 = x, y
                tile_x2, tile_y2 = x + tile_size, y + tile_size
                for cls, x1, y1, x2, y2 in abs_boxes:
                    cx = (x1 + x2) / 2
                    cy = (y1 + y2) / 2
                    if (cx >= tile_x1) and (cx <= tile_x2) and (cy >= tile_y1) and (cy <= tile_y2):
                        nx1 = max(tile_x1, x1) - tile_x1
                        ny1 = max(tile_y1, y1) - tile_y1
                        nx2 = min(tile_x2, x2) - tile_x1
                        ny2 = min(tile_y2, y2) - tile_y1
                        box_list.append((cls, nx1, ny1, nx2, ny2))
                if not box_list and not keep_neg:
                    continue
                tile = im.crop((tile_x1, tile_y1, tile_x2, tile_y2))
                tile_name = f"{stem}_tx{xi}_ty{yi}.jpg"
                tile_img_dir = os.path.join(out_dir, "images")
                tile_lbl_dir = os.path.join(out_dir, "labels")
                os.makedirs(tile_img_dir, exist_ok=True)
                os.makedirs(tile_lbl_dir, exist_ok=True)
                tile.save(os.path.join(tile_img_dir, tile_name), quality=95)
                lbl_lines = [abs_to_yolo(cls, nx1, ny1, nx2, ny2, tile_size, tile_size) for cls, nx1, ny1, nx2, ny2 in box_list]
                with open(os.path.join(tile_lbl_dir, tile_name.replace(".jpg", ".txt")), "w") as lf:
                    lf.write("\n".join(lbl_lines))

# Criar tiles para treino e teste
for img_meta in train_images:
    make_tiles_for_image(img_meta, train_root, tile_size=tile_size, overlap=overlap, keep_neg=keep_negative_tiles)
for img_meta in test_images:
    make_tiles_for_image(img_meta, test_root, tile_size=tile_size, overlap=overlap, keep_neg=True)

# Contar boxes por classe
def count_boxes_in_folder(labels_folder):
    counter = Counter()
    total = 0
    labels_path = os.path.join(labels_folder, "labels")
    if not os.path.isdir(labels_path):
        return total, dict(sorted(counter.items()))
    for f in os.listdir(labels_path):
        if not f.endswith(".txt"):
            continue
        path = os.path.join(labels_path, f)
        with open(path, "r") as fh:
            for line in fh:
                parts = line.strip().split()
                if not parts:
                    continue
                try:
                    cid = int(float(parts[0]))
                    counter[cid] += 1
                    total += 1
                except:
                    continue
    return total, dict(sorted(counter.items()))

train_img_count = len(os.listdir(os.path.join(train_root, "images"))) if os.path.isdir(os.path.join(train_root, "images")) else 0
test_img_count = len(os.listdir(os.path.join(test_root, "images"))) if os.path.isdir(os.path.join(test_root, "images")) else 0
train_total_boxes, train_dist = count_boxes_in_folder(train_root)
test_total_boxes, test_dist = count_boxes_in_folder(test_root)

names = [
    'piscina_limpa',       # 0
    'piscina_suja',        # 1
    'lona',                # 2
    'monte_de_lixo',       # 3
    'reservatorio_de_agua',# 4
    'pneu',                # 5 (era 6)
    'saco_de_lixo'         # 6 (era 7)
]
nc = len(names)

dataset_yml = {
    "train": os.path.join(train_root, "images"),
    "test": os.path.join(test_root, "images"),
    "val": os.path.join(test_root, "images"),
    "nc": nc,
    "names": names
}
dataset_yml_path = os.path.join(output_root, "dataset.yml")
with open(dataset_yml_path, "w") as yf:
    yaml.dump(dataset_yml, yf, default_flow_style=False, sort_keys=False)

# Resumo
print("random_seed:", random_seed)
print("original total images:", len(all_images))
print("train source images:", len(train_images))
print("test source images:", len(test_images))
print("train tiles:", train_img_count)
print("test tiles:", test_img_count)
print("train boxes:", train_total_boxes)
print("test boxes:", test_total_boxes)
print("train class distribution:", train_dist)
print("test class distribution:", test_dist)
print("dataset.yml:", dataset_yml_path)
print("nc:", nc)
print("names:", names)

random_seed: 42
original total images: 251
train source images: 197
test source images: 54
train tiles: 1682
test tiles: 810
train boxes: 6798
test boxes: 1643
train class distribution: {0: 112, 1: 103, 2: 745, 3: 1713, 4: 391, 5: 1742, 6: 1992}
test class distribution: {0: 14, 1: 20, 2: 208, 3: 490, 4: 117, 5: 227, 6: 567}
dataset.yml: /content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/dataset.yml
nc: 7
names: ['piscina_limpa', 'piscina_suja', 'lona', 'monte_de_lixo', 'reservatorio_de_agua', 'pneu', 'saco_de_lixo']


# Criar dataset Fine-Tuning

In [ ]:
# Fine-Tuning
import os
import re
from pathlib import Path
from PIL import Image
import sys
import shutil

SOURCE_IMAGES_DIR = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/fineTuning"
LOCAL_TILES_ROOT = "/content/FineTuning_tiles_LOCAL"
DRIVE_TILES_ROOT = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/FineTuning_tiles"

# Parâmetros de Tiling
TILE_SIZE = 1024
OVERLAP = 0.2
IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

# --- Funções Auxiliares (sem alteração) ---
def numeric_key(filename):
    """Ordena os ficheiros de forma numérica."""
    name = Path(filename).stem
    m = re.match(r"^(\d+)$", name)
    if m:
        return int(m.group(1))
    nums = re.findall(r"\d+", name)
    return int(nums[0]) if nums else name

def list_images(dirpath):
    """Lista todos os ficheiros de imagem num diretório."""
    return [f for f in os.listdir(dirpath) if Path(f).suffix.lower() in IMAGE_EXTENSIONS]

# --- Função de Tiling (v3 - Nomes Sequenciais + RGBA fix) ---

def make_hnm_tiles_for_image(img_path, out_dir, tile_size, overlap, counter_dict):
    """
    Versão v3.
    Usa um contador global para nomes sequenciais (ex: hnm_0001.jpg).
    Também converte RGBA -> RGB.
    """
    try:
        with Image.open(img_path) as im:
            W, H = im.size

            stride = int(tile_size * (1 - overlap))

            x_starts = list(range(0, max(1, W - tile_size + 1), stride))
            y_starts = list(range(0, max(1, H - tile_size + 1), stride))

            if W > tile_size and (not x_starts or x_starts[-1] + tile_size < W):
                 x_starts.append(W - tile_size)
            if H > tile_size and (not y_starts or y_starts[-1] + tile_size < H):
                 y_starts.append(H - tile_size)

            if not x_starts:
                x_starts = [0]
            if not y_starts:
                y_starts = [0]

            tile_count = 0
            for xi, x in enumerate(x_starts):
                for yi, y in enumerate(y_starts):
                    tile_x1, tile_y1 = x, y
                    tile_x2, tile_y2 = x + tile_size, y + tile_size

                    tile = im.crop((tile_x1, tile_y1, tile_x2, tile_y2))

                    if tile.width == 0 or tile.height == 0:
                        continue

                    # Lógica do Nome do Ficheiro Sequencial
                    tile_name = f"hnm_{counter_dict['value']:04d}.jpg"
                    counter_dict['value'] += 1

                    tile_img_dir = os.path.join(out_dir, "images")
                    os.makedirs(tile_img_dir, exist_ok=True)

                    if tile.mode == 'RGBA':
                        tile = tile.convert('RGB')

                    tile.save(os.path.join(tile_img_dir, tile_name), quality=95)
                    tile_count += 1

            return tile_count

    except Exception as e:
        print(f"  [ERRO] Falha ao processar {img_path}: {e}")
        return 0

# --- Script Principal (Escrita Local + Cópia para o Drive) ---
def main():
    print(f"Iniciando processo de tiling para Hard Negative Mining (HNM)...")
    print(f"Pasta de origem: {SOURCE_IMAGES_DIR}")
    print(f"Pasta de destino LOCAL: {LOCAL_TILES_ROOT}")

    # Limpa a pasta local, se já existir
    if os.path.exists(LOCAL_TILES_ROOT):
        print(f"Limpando pasta local antiga: {LOCAL_TILES_ROOT}")
        shutil.rmtree(LOCAL_TILES_ROOT)

    if not os.path.isdir(SOURCE_IMAGES_DIR):
        print(f"\n[ERRO FATAL] A pasta de origem não existe: {SOURCE_IMAGES_DIR}")
        sys.exit(1)

    all_images = sorted(list_images(SOURCE_IMAGES_DIR), key=numeric_key)

    if not all_images:
        print(f"\n[AVISO] Nenhuma imagem encontrada em {SOURCE_IMAGES_DIR}")
        return

    # Inicializa o contador global
    global_tile_counter = {"value": 1}

    total_tiles_created = 0
    print(f"\nEncontradas {len(all_images)} imagens-fonte. Começando o tiling (local)...")

    for img_name in all_images:
        img_path = os.path.join(SOURCE_IMAGES_DIR, img_name)

        # Salva na pasta LOCAL_TILES_ROOT
        count = make_hnm_tiles_for_image(
            img_path,
            LOCAL_TILES_ROOT,
            tile_size=TILE_SIZE,
            overlap=OVERLAP,
            counter_dict=global_tile_counter  # Passa o contador
        )
        if count > 0:
            print(f"Processando {img_name}... -> Criados {count} tiles.")
        total_tiles_created += count

    print("\n--- Tiling Local Concluído ---")
    print(f"Total de tiles de HNM criados localmente: {total_tiles_created}")

    # Verifica a contagem local ANTES de copiar
    local_img_folder = os.path.join(LOCAL_TILES_ROOT, "images")
    if os.path.exists(local_img_folder):
        real_count = len(os.listdir(local_img_folder))
        print(f"Verificação local: {real_count} tiles existem na pasta local.")

        if real_count == total_tiles_created and real_count > 0:
            # Copia tudo de uma vez para o Google Drive
            print(f"\nCopiando {real_count} tiles para o Google Drive...")

            drive_img_folder = os.path.join(DRIVE_TILES_ROOT, "images")

            # Limpa a pasta de destino do Drive, se existir
            if os.path.exists(drive_img_folder):
                print(f"Limpando pasta de destino antiga no Drive: {drive_img_folder}")
                shutil.rmtree(drive_img_folder)

            print(f"De: {local_img_folder}")
            print(f"Para: {drive_img_folder}")

            shutil.copytree(local_img_folder, drive_img_folder)
            print("Cópia para o Google Drive concluída!")

            print("\n[PRÓXIMO PASSO]")
            print("Copie todos os ficheiros de:")
            print(f"'{drive_img_folder}'")
            print("Para a sua pasta de treino original:")
            print(f"'{os.path.join(output_root, 'tiles', 'train', 'images')}'")
        else:
            print(f"[ERRO] A contagem local ({real_count}) não bate com a esperada ({total_tiles_created}).")
    else:
         print("[ERRO] A pasta local de tiles não foi criada.")


if __name__ == "__main__":
    output_root = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2"
    main()

Iniciando processo de tiling para Hard Negative Mining (HNM)...
Pasta de origem: /content/drive/MyDrive/tcc_dengue/ia_deteccao/fineTuning
Pasta de destino LOCAL: /content/FineTuning_tiles_LOCAL

Encontradas 64 imagens-fonte. Começando o tiling (local)...
Processando 2.png... -> Criados 2 tiles.
Processando 4.png... -> Criados 2 tiles.
Processando 8.png... -> Criados 2 tiles.
Processando Captura de tela 2025-10-17 210203.png... -> Criados 2 tiles.
Processando Captura de tela 2025-10-17 210922.png... -> Criados 2 tiles.
Processando Captura de tela 2025-10-17 210956.png... -> Criados 2 tiles.
Processando Captura de tela 2025-10-17 211022.png... -> Criados 2 tiles.
Processando Captura de tela 2025-10-17 211053.png... -> Criados 2 tiles.
Processando Captura de tela 2025-10-17 211200.png... -> Criados 2 tiles.
Processando Captura de tela 2025-10-17 211213.png... -> Criados 2 tiles.
Processando Captura de tela 2025-10-17 211256.png... -> Criados 2 tiles.
Processando Captura de tela 2025-10-17

In [ ]:
# Recuperar Backgrounds de treino

import os
import re
import shutil
import random
from pathlib import Path
from collections import Counter, defaultdict
from PIL import Image
import sys

# --- 1. CONFIGURAÇÃO ---
images_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Sao_Carlos_imagens"
labels_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Sao_Carlos_finalAnnotations"

# Destino LOCAL (para velocidade e segurança)
LOCAL_OUTPUT_ROOT = "/content/Discarded_Backgrounds_LOCAL"

# Destino FINAL no seu Drive
DRIVE_OUTPUT_ROOT = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Discarded_Backgrounds"

tile_size = 1024
overlap = 0.2
random_seed = 42
random.seed(random_seed)
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

def numeric_key(filename):
    name = Path(filename).stem
    m = re.match(r"^(\d+)$", name)
    if m:
        return int(m.group(1))
    nums = re.findall(r"\d+", name)
    return int(nums[0]) if nums else name

def list_images(dirpath):
    return [f for f in os.listdir(dirpath) if Path(f).suffix.lower() in image_extensions]

def ensure_label_file(labels_dir, stem):
    src = os.path.join(labels_dir, stem + ".txt")
    if not os.path.exists(src):
        open(src, "a").close()
    return src

def read_yolo_labels(path):
    boxes = []
    try:
        with open(path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if not parts: continue
                cls = int(float(parts[0]))
                if cls in [5, 8]: continue
                if cls > 5: cls -= 1
                boxes.append((cls, float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])))
    except:
        pass
    return boxes

def yolo_to_abs(box, img_w, img_h):
    cls, x_c, y_c, w, h = box
    x_c *= img_w; y_c *= img_h; w *= img_w; h *= img_h
    x1 = x_c - w / 2; y1 = y_c - h / 2
    x2 = x_c + w / 2; y2 = y_c + h / 2
    return cls, x1, y1, x2, y2

def dominant_class_for_image(boxes):
    if not boxes: return -1
    counter = Counter([b[0] for b in boxes])
    return counter.most_common(1)[0][0]

# --- 3. FUNÇÃO DE TILING MODIFICADA (A Lógica Invertida) ---

def save_negative_tiles_only(meta, out_dir, tile_size, overlap, counter_dict):
    """
    Função modificada para salvar APENAS tiles de background.
    """
    img_name = meta["img"]
    img_path = os.path.join(images_dir, img_name)
    abs_boxes = [yolo_to_abs(b, meta["w"], meta["h"]) for b in meta["boxes"]]
    stride = int(tile_size * (1 - overlap))

    tile_count = 0

    try:
        with Image.open(img_path) as im:
            W, H = im.size
            x_starts = list(range(0, max(1, W - tile_size + 1), stride))
            y_starts = list(range(0, max(1, H - tile_size + 1), stride))

            if W > tile_size and (not x_starts or x_starts[-1] + tile_size < W):
                 x_starts.append(W - tile_size)
            if H > tile_size and (not y_starts or y_starts[-1] + tile_size < H):
                 y_starts.append(H - tile_size)

            if not x_starts: x_starts = [0]
            if not y_starts: y_starts = [0]

            for xi, x in enumerate(x_starts):
                for yi, y in enumerate(y_starts):
                    tile_x1, tile_y1 = x, y
                    tile_x2, tile_y2 = x + tile_size, y + tile_size

                    # Verifica se algum label está no tile
                    box_list = []
                    for cls, x1, y1, x2, y2 in abs_boxes:
                        cx = (x1 + x2) / 2
                        cy = (y1 + y2) / 2
                        if (cx >= tile_x1) and (cx <= tile_x2) and (cy >= tile_y1) and (cy <= tile_y2):
                            box_list.append(True) # Só precisamos saber se tem *algum*
                            break # Otimização: achou um, pode parar

                    # --- A LÓGICA INVERTIDA ---
                    if not box_list:
                        # Se a lista de caixas (box_list) está VAZIA, é um background. SALVE!
                        tile = im.crop((tile_x1, tile_y1, tile_x2, tile_y2))

                        if tile.width == 0 or tile.height == 0: continue

                        # Nome sequencial
                        tile_name = f"bg_{counter_dict['value']:05d}.jpg" # 5 dígitos para ~1600+
                        counter_dict['value'] += 1

                        tile_img_dir = os.path.join(out_dir, "images")
                        os.makedirs(tile_img_dir, exist_ok=True)

                        if tile.mode == 'RGBA':
                            tile = tile.convert('RGB')

                        tile.save(os.path.join(tile_img_dir, tile_name), quality=95)
                        tile_count += 1
                    else:
                        # Se a lista NÃO está vazia, é um tile positivo. IGNORE.
                        continue
        return tile_count

    except Exception as e:
        print(f"  [ERRO] Falha ao processar {img_name}: {e}")
        return 0

# --- 4. SCRIPT PRINCIPAL (Modificado) ---
def main():
    print(f"Iniciando recuperação de backgrounds descartados...")
    print(f"Usando random_seed = {random_seed} para replicar o split de treino.")

    all_images = sorted(list_images(images_dir), key=numeric_key)
    if not all_images:
        raise SystemExit("Nenhuma imagem encontrada em images_dir")

    # --- PASSO A: Replicar o split 80/20 original ---
    print("Coletando metadados das imagens-fonte...")
    image_meta = []
    for img_name in all_images:
        stem = Path(img_name).stem
        lbl = ensure_label_file(labels_dir, stem)
        img_path = os.path.join(images_dir, img_name)
        try:
            with Image.open(img_path) as im:
                w, h = im.size
        except:
            print(f"Aviso: Falha ao abrir {img_path}, pulando.")
            continue
        boxes = read_yolo_labels(lbl)
        dom = dominant_class_for_image(boxes)
        image_meta.append({"img": img_name, "stem": stem, "w": w, "h": h, "boxes": boxes, "dom": dom})

    print("Replicando o split de treino estratificado 80/20...")
    groups = defaultdict(list)
    for m in image_meta:
        groups[m["dom"]].append(m)

    train_images = []
    test_images = [] # Não vamos usar, mas mantemos para a lógica ser idêntica
    for dom, items in groups.items():
        random.shuffle(items) # <-- Graças ao seed 42, isto é idêntico
        split = int(0.8 * len(items))
        train_images.extend(items[:split])
        test_images.extend(items[split:])

    print(f"Split replicado. {len(train_images)} imagens-fonte de treino encontradas.")

    # --- PASSO B: Fazer o Tiling e Salvar (Localmente) ---

    # Limpa a pasta local, se já existir
    if os.path.exists(LOCAL_OUTPUT_ROOT):
        print(f"Limpando pasta local antiga: {LOCAL_OUTPUT_ROOT}")
        shutil.rmtree(LOCAL_OUTPUT_ROOT)

    global_tile_counter = {"value": 1}
    total_tiles_created = 0
    print(f"Começando o tiling (local) para salvar APENAS backgrounds...")

    for img_meta in train_images: # <-- Itera APENAS nas imagens de treino
        count = save_negative_tiles_only(
            img_meta,
            LOCAL_OUTPUT_ROOT,
            tile_size=tile_size,
            overlap=overlap,
            counter_dict=global_tile_counter
        )
        if count > 0:
            print(f"Processando {img_meta['img']}... -> Salvos {count} tiles de background.")
        total_tiles_created += count

    print("\n--- Tiling Local Concluído ---")
    print(f"Total de tiles de background recuperados: {total_tiles_created}")

    # --- PASSO C: Verificar e Copiar para o Drive ---
    local_img_folder = os.path.join(LOCAL_OUTPUT_ROOT, "images")
    if os.path.exists(local_img_folder):
        real_count = len(os.listdir(local_img_folder))
        print(f"Verificação local: {real_count} tiles existem na pasta local.")

        if real_count == total_tiles_created and real_count > 0:
            print(f"\nCopiando {real_count} tiles para o Google Drive...")

            drive_img_folder = os.path.join(DRIVE_OUTPUT_ROOT, "images")

            if os.path.exists(drive_img_folder):
                print(f"Limpando pasta de destino antiga no Drive: {drive_img_folder}")
                shutil.rmtree(drive_img_folder)

            print(f"De: {local_img_folder}")
            print(f"Para: {drive_img_folder}")

            shutil.copytree(local_img_folder, drive_img_folder)
            print("Cópia para o Google Drive concluída!")

            print("\n[PRÓXIMO PASSO]")
            print(f"Adicione os {real_count} ficheiros de:")
            print(f"'{drive_img_folder}'")
            print("Para a sua pasta de treino original:")
            print("'/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/tiles/train/images'")
        else:
            print(f"[ERRO] A contagem local ({real_count}) não bate com a esperada ({total_tiles_created}).")
    else:
         print("[ERRO] A pasta local de tiles não foi criada.")


if __name__ == "__main__":
    main()

Iniciando recuperação de backgrounds descartados...
Usando random_seed = 42 para replicar o split de treino.
Coletando metadados das imagens-fonte...
Replicando o split de treino estratificado 80/20...
Split replicado. 197 imagens-fonte de treino encontradas.
Começando o tiling (local) para salvar APENAS backgrounds...
Processando 59.jpg... -> Salvos 4 tiles de background.
Processando 120.jpg... -> Salvos 12 tiles de background.
Processando 145.jpg... -> Salvos 8 tiles de background.
Processando 27.jpg... -> Salvos 2 tiles de background.
Processando 114.jpg... -> Salvos 4 tiles de background.
Processando 182.jpg... -> Salvos 6 tiles de background.
Processando 96.jpg... -> Salvos 6 tiles de background.
Processando 186.jpg... -> Salvos 2 tiles de background.
Processando 100.jpg... -> Salvos 3 tiles de background.
Processando 73.jpg... -> Salvos 7 tiles de background.
Processando 231.jpg... -> Salvos 8 tiles de background.
Processando 131.jpg... -> Salvos 7 tiles de background.
Processand

In [ ]:
import os
import shutil
from pathlib import Path

SOURCE_DIR = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Discarded_Backgrounds/images"
DESTINATION_DIR = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/tiles/train/images"
print("Iniciando transferência de ficheiros de background...")

if not os.path.exists(SOURCE_DIR):
    print(f"[ERRO FATAL] A pasta de origem não existe: {SOURCE_DIR}")
    sys.exit(1)
if not os.path.exists(DESTINATION_DIR):
    print(f"[ERRO FATAL] A pasta de destino não existe: {DESTINATION_DIR}")
    sys.exit(1)

files_to_copy = [f for f in os.listdir(SOURCE_DIR) if f.endswith('.jpg')]
num_to_copy = len(files_to_copy)

if num_to_copy == 0:
    print(f"[ERRO] Nenhum ficheiro .jpg encontrado em {SOURCE_DIR}")
    sys.exit(1)

try:
    initial_count = len([f for f in os.listdir(DESTINATION_DIR) if f.endswith('.jpg')])
except Exception as e:
    print(f"[ERRO] Não foi possível ler a pasta de destino: {e}")
    sys.exit(1)

print(f"Pasta de origem ({SOURCE_DIR}) contém: {num_to_copy} ficheiros.")
print(f"Pasta de destino ({DESTINATION_DIR}) contém: {initial_count} ficheiros.")
print("\nIniciando a cópia...")

for i, filename in enumerate(files_to_copy):
    source_path = os.path.join(SOURCE_DIR, filename)
    destination_path = os.path.join(DESTINATION_DIR, filename)

    if not os.path.exists(destination_path):
        shutil.copy2(source_path, destination_path)

    if (i + 1) % 25 == 0 or (i + 1) == num_to_copy:
        print(f"Progresso: {i + 1} / {num_to_copy} copiados.", end='\r')

print("\nCópia concluída.")

# --- 4. VERIFICAÇÃO FINAL ---
print("Verificando a contagem final...")
try:
    final_count = len([f for f in os.listdir(DESTINATION_DIR) if f.endswith('.jpg')])
except Exception as e:
    print(f"[ERRO] Não foi possível ler a pasta de destino após a cópia: {e}")
    sys.exit(1)

expected_total_count = initial_count + num_to_copy

print(f"Contagem inicial: {initial_count}")
print(f"Ficheiros adicionados: {num_to_copy}")
print(f"Contagem final na pasta: {final_count}")
print(f"Contagem total esperada: {expected_total_count}")

if final_count == expected_total_count:
    print("\n✅ SUCESSO! A contagem de ficheiros está correta.")
    print("O seu dataset de treino está pronto para o fine-tuning final.")
else:
    print(f"\n[AVISO!] A contagem final ({final_count}) não bate com a esperada ({expected_total_count}).")
    print("Pode haver ficheiros duplicados ou um erro de cópia. Verifique as pastas.")

Iniciando transferência de ficheiros de background...
Pasta de origem (/content/drive/MyDrive/tcc_dengue/ia_deteccao/Discarded_Backgrounds/images) contém: 1273 ficheiros.
Pasta de destino (/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/tiles/train/images) contém: 1810 ficheiros.

Iniciando a cópia...
Progresso: 1273 / 1273 copiados.
Cópia concluída.
Verificando a contagem final...
Contagem inicial: 1810
Ficheiros adicionados: 1273
Contagem final na pasta: 3083
Contagem total esperada: 3083

✅ SUCESSO! A contagem de ficheiros está correta.
O seu dataset de treino está pronto para o fine-tuning final.


# Treinando

In [4]:
from ultralytics import YOLO
from pathlib import Path

dataset_yaml_path = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/dataset.yml"
project_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/modelo_final/runs"
old_run_name = "dengue_detection_testeI11"
pretrained_best = Path(f"{project_dir}/{old_run_name}/weights/best.pt")

run_name = "tiled_yolov8m_tilesFINAL"

train_args = {
    "data": dataset_yaml_path,
    "epochs": 100,
    "imgsz": 1024,
    "project": project_dir,
    "name": run_name,
    "save": True,
    "save_period": 30,
    "batch": 8,
    "workers": 6,
    "patience": 30,
    "exist_ok": True,
    "rect": False,
    "optimizer": "AdamW",
    "lr0": 0.00015,
    "freeze": 10,
    "lrf": 0.01,
    "weight_decay": 0.002,
    "seed": 42,
    "hsv_h": 0.03,
    "hsv_s": 0.5,
    "hsv_v": 0.4,
    "degrees": 10.0,
    "translate": 0.1,
    "scale": 0.1,
    "fliplr": 0.5,
    "flipud": 0.0,
    "mosaic": 0.2,
    "mixup": 0.1,
    "copy_paste": 0.1,
    "erasing": 0.0,
}

if pretrained_best.exists():
    model = YOLO(str(pretrained_best))
    model.train(**train_args)
else:
    model = YOLO("yolov8m.pt")
    model.train(**train_args)


Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/dataset.yml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.0, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.03, hsv_s=0.5, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00015, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/modelo_final/runs/dengue_detection_testeI11/weights/best.pt, momentum=0.937, mosaic=0.2, multi_s

KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import os

dataset_yaml_path = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/dataset.yml"
project_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/modelo_final/runs"

# Modelo BASE para transfer learning (usado apenas se for um NOVO treino)
source_run_name = "tiled_yolo8m_fineTuned"
model_to_fine_tune = Path(f"{project_dir}/{source_run_name}/weights/best.pt")

target_run_name = "tiled_yolo8m_fineTunedBackGrounds"
resume_checkpoint = Path(f"{project_dir}/{target_run_name}/weights/last.pt")

fine_tune_args = {
    "data": dataset_yaml_path,
    "lr0": 1e-5,
    "epochs": 50,
    "patience": 20,
    "freeze": 0,
    "mosaic": 0.0,
    "mixup": 0.0,
    "copy_paste": 0.0,
    "imgsz": 1024,
    "project": project_dir,
    "name": target_run_name,
    "batch": 8,
    "workers": 6,
    "exist_ok": True,
    "optimizer": "AdamW",
    "hsv_h": 0.03,
    "hsv_s": 0.5,
    "hsv_v": 0.4,
    "degrees": 10.0,
    "translate": 0.1,
    "scale": 0.1,
    "fliplr": 0.5,
}

# Lógica de Resumo: Verifica se o 'last.pt' da run ALVO existe
if resume_checkpoint.exists():
    print(f"Checkpoint 'last.pt' encontrado em: {resume_checkpoint}")
    print(f"Continuando (resumindo) o treino da run '{target_run_name}'...")
    model = YOLO(str(resume_checkpoint))
    model.train(resume=True)

    print("Treino (resumido) concluído.")

# Lógica de Novo Treino: Se 'last.pt' não existe, começa um novo fine-tuning
else:
    print(f"Nenhum checkpoint 'last.pt' encontrado em '{target_run_name}'.")
    print(f"Verificando modelo base para novo fine-tuning...")
    if not model_to_fine_tune.exists():
        print(f"[ERRO FATAL] O modelo base 'best.pt' para fine-tuning não foi encontrado em:")
        print(f"{model_to_fine_tune}")

    else:
        print(f"Iniciando NOVO fine-tuning (HNM)...")
        print(f"Carregando modelo base de: {model_to_fine_tune}")
        print(f"Salvando nova run em: {project_dir}/{target_run_name}")
        model = YOLO(str(model_to_fine_tune))
        model.train(**fine_tune_args)

        print("Fine-tuning (novo) concluído.")

Checkpoint 'last.pt' encontrado em: /content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/modelo_final/runs/tiled_yolo8m_fineTunedBackGrounds/weights/last.pt
Continuando (resumindo) o treino da run 'tiled_yolo8m_fineTunedBackGrounds'...
Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/dataset.yml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.03, hsv_s=0.5, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=

# Testando

In [ ]:
import os
run_name = "dengue_detection_testeI11"
run_name = "tiled_yolov8m_tilesFINAL"

base_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2"
model_dir = os.path.join(base_dir, "modelo_final", "runs", run_name, "weights", "best.pt")
dataset_yaml_path = os.path.join(base_dir, "dataset.yml")

model = YOLO(model_dir)
metrics = model.val(data=dataset_yaml_path)

mean_precision, mean_recall, map50, map50_95 = metrics.mean_results()
print("=== Métricas gerais ===")
print(f"Mean Precision: {mean_precision:.3f}")
print(f"Mean Recall: {mean_recall:.3f}")
print(f"mAP@0.5: {map50:.3f}")
print(f"mAP@0.5:0.95: {map50_95:.3f}")

Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,843,813 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.8±0.2 ms, read: 0.5±0.1 MB/s, size: 432.1 KB)
val: Scanning /content/drive/.shortcut-targets-by-id/1iqKFMZvfF0pgK9N8REMQ7NqepaG-Pnv3/tcc_dengue/ia_deteccao/Teste_Treino_2/tiles/test/labels.cache... 810 images, 403 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 810/810 1.2Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 51/51 0.8it/s 1:05
                   all        810       1643      0.768      0.707      0.762      0.535
         piscina_limpa         14         14      0.667      0.857      0.838      0.666
          piscina_suja         20         20      0.667       0.75      0.802      0.674
                  lona        139        208      0.864      0.812      0.825      0.627
         monte_de_lixo        261       

# Rascunho

In [ ]:
'''from pathlib import Path

dataset_yaml_path = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/modelo_final/dataset.yaml"
project_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/modelo_final/runs"
run_name = "dengue_detectionSmall"

# Verifica se existe um modelo anterior salvo
resume_path = Path(f"{project_dir}/{run_name}/weights/last.pt")

if resume_path.exists():
    print("✅ Retomando treino do último checkpoint...")
    model = YOLO(f"{resume_path}")
    model.train(
        resume=True,
        exist_ok=True,
        data=dataset_yaml_path,
        epochs=80, # Para o pro: 150
        imgsz=960, #antes: 600
        project=project_dir,
        name=run_name,
        save=True,
        save_period=5, # Salvar a cada 5 épocas
        batch=16,  # Para o Pro: 32
        workers=2, # Para o Pro: 4
        patience=20,  # se overfitting, para cedo, para o Pro: 35
    )
else:
    print("🆕 Iniciando novo treino do zero...")
    model = YOLO("yolov8s.pt") # Para o Pro: "yolov8m.pt"
    model.train(
        data=dataset_yaml_path,
        epochs=80, # Para o pro: 150
        imgsz=640,
        project=project_dir,
        name=run_name,
        save=True,
        save_period=5, # Salvar a cada 5 épocas
        batch=16,  # Para o Pro: 32
        workers=2, # Para o Pro: 4
        patience=20,  # se overfitting, para cedo, para o Pro: 35
    )
'''

In [ ]:
import os
import re
import shutil
import random
from pathlib import Path
from collections import Counter
import yaml

images_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Sao_Carlos_imagens"
labels_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Sao_Carlos_finalAnnotations"
output_root = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2"
train_img_dir = os.path.join(output_root, "train", "images")
train_lbl_dir = os.path.join(output_root, "train", "labels")
test_img_dir = os.path.join(output_root, "test", "images")
test_lbl_dir = os.path.join(output_root, "test", "labels")
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(test_img_dir, exist_ok=True)
os.makedirs(test_lbl_dir, exist_ok=True)

random_seed = 42
random.seed(random_seed)

# --- Funções Utilitárias (sem alterações) ---
def numeric_key(filename):
    name = Path(filename).stem
    m = re.match(r"^(\d+)$", name)
    if m:
        return int(m.group(1))
    nums = re.findall(r"\d+", name)
    return int(nums[0]) if nums else name

image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
all_images = [f for f in os.listdir(images_dir) if Path(f).suffix.lower() in image_extensions]
if not all_images:
    raise SystemExit("no images found in images_dir")
all_images_sorted = sorted(all_images, key=numeric_key)
random.shuffle(all_images_sorted)
total = len(all_images_sorted)
split_idx = int(0.8 * total)
train_images = all_images_sorted[:split_idx]
test_images = all_images_sorted[split_idx:]

def ensure_label_exists(labels_dir, base_stem):
    src = os.path.join(labels_dir, base_stem + ".txt")
    if not os.path.exists(src):
        open(src, "a").close()
    return src


CLASSES_TO_REMOVE = {5, 8}

CLASS_REMAPPING = {
    6: 5,
    7: 6
}

def process_and_copy_label(src_path, dst_path):
    """Lê um arquivo de anotação, remove/re-indexa classes e salva no destino."""
    new_lines = []
    with open(src_path, "r") as f_in:
        for line in f_in:
            parts = line.strip().split()
            if not parts:
                continue

            try:
                original_cls = int(float(parts[0]))

                if original_cls in CLASSES_TO_REMOVE:
                    continue

                new_cls = CLASS_REMAPPING.get(original_cls, original_cls)

                new_line = f"{new_cls} {' '.join(parts[1:])}"
                new_lines.append(new_line)
            except ValueError:
                continue

    with open(dst_path, "w") as f_out:
        f_out.write("\n".join(new_lines))


# --- Processamento e Cópia dos Arquivos ---
for img_list, dest_img_dir, dest_lbl_dir in [
    (train_images, train_img_dir, train_lbl_dir),
    (test_images, test_img_dir, test_lbl_dir),
]:
    for img_name in img_list:
        src_img = os.path.join(images_dir, img_name)
        dst_img = os.path.join(dest_img_dir, img_name)
        shutil.copy2(src_img, dst_img)

        base = Path(img_name).stem
        lbl_src = ensure_label_exists(labels_dir, base)
        dst_lbl = os.path.join(dest_lbl_dir, base + ".txt")
        process_and_copy_label(lbl_src, dst_lbl)

train_counter = Counter()
test_counter = Counter()
def count_classes_in_dir(labels_dir, counter):
    total_boxes = 0
    for fname in os.listdir(labels_dir):
        if not fname.endswith(".txt"):
            continue
        path = os.path.join(labels_dir, fname)
        with open(path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue
                try:
                    cid = int(float(parts[0]))
                    counter[cid] += 1
                    total_boxes += 1
                except:
                    continue
    return total_boxes

train_total_boxes = count_classes_in_dir(train_lbl_dir, train_counter)
test_total_boxes = count_classes_in_dir(test_lbl_dir, test_counter)

names = [
    'piscina_limpa',        # 0
    'piscina_suja',         # 1
    'lona',                 # 2
    'monte_de_lixo',        # 3
    'reservatorio_de_agua', # 4
    'pneu',                 # 5 (era 6)
    'saco_de_lixo'          # 6 (era 7)
]
nc = len(names)

dataset_yml = {
    "train": train_img_dir,
    "val": test_img_dir,
    "test": test_img_dir,
    "nc": nc,
    "names": names
}

dataset_yml_path = os.path.join(output_root, "dataset.yml")
with open(dataset_yml_path, "w") as yf:
    yaml.dump(dataset_yml, yf, default_flow_style=False, sort_keys=False)

print("random_seed:", random_seed)
print("total images:", total)
print("train images:", len(train_images))
print("test images:", len(test_images))
print("train boxes:", train_total_boxes)
print("test boxes:", test_total_boxes)
print("train class distribution:", dict(sorted(train_counter.items())))
print("test class distribution:", dict(sorted(test_counter.items())))
print("dataset.yml:", dataset_yml_path)
print("nc:", nc)
print("names:", names)

random_seed: 42
total images: 251
train images: 200
test images: 51
train boxes: 4035
test boxes: 841
train class distribution: {0: 59, 1: 59, 2: 424, 3: 1037, 4: 215, 5: 1070, 6: 1171}
test class distribution: {0: 15, 1: 14, 2: 115, 3: 229, 4: 60, 5: 175, 6: 233}
dataset.yml: /content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/dataset.yml
nc: 7
names: ['piscina_limpa', 'piscina_suja', 'lona', 'monte_de_lixo', 'reservatorio_de_agua', 'pneu', 'saco_de_lixo']


In [ ]:
from ultralytics import YOLO
from pathlib import Path

# --- CONFIGURAÇÃO DE CAMINHOS ---
dataset_yaml_path = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/dataset.yml"
project_dir = "/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/modelo_final/runs"
run_name = "dengue_detection_50porcento3"

# Verifica se existe um modelo anterior salvo
resume_path = Path(f"{project_dir}/{run_name}/weights/last.pt")

# --- ARGUMENTOS DE TREINAMENTO ---
train_args = {
    "data": dataset_yaml_path,
    "epochs": 120,
    "imgsz": 1408,
    "project": project_dir,
    "name": run_name,
    "save": True,
    "save_period": 15,
    "batch": 4,
    "workers": 6,
    "patience": 30,
    "exist_ok": True,
    "rect": True,
    "optimizer": "SGD",
    "lr0": 0.01,
    "lrf": 0.01,
    "weight_decay": 0.0005,

    "degrees": 10.0,
    "translate": 0.1,
    "scale": 0.1,
    "fliplr": 0.5,
    "mosaic": 1.0,
    "mixup": 0.1,
    "copy_paste": 0.1,
    "cache": "ram"
}

if resume_path.exists():
    print("✅ Retomando treino do último checkpoint...")
    model = YOLO(f"{resume_path}")
    model.train(resume=True, **train_args)
else:
    print("🆕 Iniciando novo treino do zero com YOLOv8m...")
    model = YOLO("yolov8m.pt")
    model.train(**train_args)


🆕 Iniciando novo treino do zero com YOLOv8m...
Ultralytics 8.3.215 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/tcc_dengue/ia_deteccao/Teste_Treino_2/dataset.yml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1408, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=dengue_detection_50porcento3, nbs=64, nms=False,